In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [4]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.992275976305792,
           'expected_sample_data_size': 16.513548549720028,
           'expected_time_to_recover': 99.0899,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16146953423103666,
             'bit_redundancy': 415.4495,
             'ideal_bit_efficency': 0.1907280189673781,
             'ideal_bit_redundancy': 339.4454513454738,
             'ideal_packet_efficiency': 0.9536400948368905,
             'ideal_packet_redundancy': 67.88909026909477,
             'ideal_time_to_recover': 83.88909026909477,
             'ideal_transmitted_bits': 419.4454513454738,
             'packet_efficiency': 0.8073476711551834,
             'packet_redundancy': 83.0899,
             'permeability': 0.8465957708010077,
             'saturation': 16.0,
             'transmitted_bits': 495.4495}}
Found implementation: anfext_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.0044628038210455,
           'expected_sample_data_size': 16.542802345738206,
           'expected_time_to_recover': 96.71809999999999,
           'packet_size': 5,
           'payload_bits': 78.25340432877415},
 'derived': {'bit_efficency': 0.16181749709469925,
             'bit_redundancy': 405.3370956712258,
             'ideal_bit_efficency': 0.19120639027419695,
             'ideal_bit_redundancy': 331.0080446037428,
             'ideal_packet_efficiency': 0.9560319513709846,
             'ideal_packet_redundancy': 66.20160892074855,
             'ideal_time_to_recover': 81.85228978650339,
             'ideal_transmitted_bits': 409.26144893251694,
             'packet_efficiency': 0.8090874854734963,
             'packet_redundancy': 81.06741913424516,
             'permeability': 0.8462975367227374,
             'saturation': 15.65068086575483,
             'transmitted_bits': 483.59049999999996}}
Found implementation: anfext_fullrank2
 

benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1829.67it/s]


{'basic': {'expected_sample_burst_size': 2.9948178202707587,
           'expected_sample_data_size': 16.48491543807496,
           'expected_time_to_recover': 39.2748,
           'packet_size': 5,
           'payload_bits': 78.25340432877415},
 'derived': {'bit_efficency': 0.3984916757247607,
             'bit_redundancy': 118.12059567122584,
             'ideal_bit_efficency': 0.4708857365965361,
             'ideal_bit_redundancy': 87.93002032616111,
             'ideal_packet_efficiency': 2.3544286829826806,
             'ideal_packet_redundancy': 17.586004065232224,
             'ideal_time_to_recover': 33.23668493098705,
             'ideal_transmitted_bits': 166.18342465493527,
             'packet_efficiency': 1.9924583786238035,
             'packet_redundancy': 23.62411913424517,
             'permeability': 0.846259813697003,
             'saturation': 15.65068086575483,
             'transmitted_bits': 196.374}}
Found implementation: shuffled_binary_fullrank
  No changes. Us

{'basic': {'expected_sample_burst_size': 2.992846590467304,
           'expected_sample_data_size': 16.36544592177902,
           'expected_time_to_recover': 14.1254,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3292424190291836,
             'bit_redundancy': 47.37359567122586,
             'ideal_bit_efficency': 0.38945294161062904,
             'ideal_bit_redundancy': 36.45446238448504,
             'ideal_packet_efficiency': 1.9472647080531453,
             'ideal_packet_redundancy': 7.290892476897007,
             'ideal_time_to_recover': 11.941573342651838,
             'ideal_transmitted_bits': 59.70786671325919,
             'packet_efficiency': 1.646212095145918,
             'packet_redundancy': 9.47471913424517,
             'permeability': 0.8453971811525222,
             'saturation': 4.650680865754831,
             'transmitted_bits': 70.62700000000001}}
Found implementation: simple_matrix
  No changes. Using

{'basic': {'expected_sample_burst_size': 2.994337742344628,
           'expected_sample_data_size': 16.518798695415835,
           'expected_time_to_recover': 32.790800000000004,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15248179367383533,
             'bit_redundancy': 138.954,
             'ideal_bit_efficency': 0.18012193859216274,
             'ideal_bit_redundancy': 113.7948641648018,
             'ideal_packet_efficiency': 0.9006096929608136,
             'ideal_packet_redundancy': 22.758972832960364,
             'ideal_time_to_recover': 27.758972832960364,
             'ideal_transmitted_bits': 138.7948641648018,
             'packet_efficiency': 0.7624089683691766,
             'packet_redundancy': 27.790800000000004,
             'permeability': 0.8465475936226124,
             'saturation': 5.0,
             'transmitted_bits': 163.954}}
Found implementation: single_binary_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.993641204991654,
           'expected_sample_data_size': 16.442810587393687,
           'expected_time_to_recover': 13.2018,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3522762703385016,
             'bit_redundancy': 42.75559567122585,
             'ideal_bit_efficency': 0.41641304019429776,
             'ideal_bit_redundancy': 32.588757381445454,
             'ideal_packet_efficiency': 2.082065200971489,
             'ideal_packet_redundancy': 6.5177514762890905,
             'ideal_time_to_recover': 11.168432342043921,
             'ideal_transmitted_bits': 55.84216171021961,
             'packet_efficiency': 1.7613813516925079,
             'packet_redundancy': 8.55111913424517,
             'permeability': 0.8459779986095776,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.009}}
Found implementation: single_binary_fullrank2
  No changes. Usi

{'basic': {'expected_sample_burst_size': 3.0018111662335616,
           'expected_sample_data_size': 16.482793920781162,
           'expected_time_to_recover': 13.8567,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3356268711709736,
             'bit_redundancy': 46.03009567122585,
             'ideal_bit_efficency': 0.39675051892215013,
             'ideal_bit_redundancy': 35.35623376809978,
             'ideal_packet_efficiency': 1.9837525946107508,
             'ideal_packet_redundancy': 7.071246753619954,
             'ideal_time_to_recover': 11.721927619374785,
             'ideal_transmitted_bits': 58.60963809687393,
             'packet_efficiency': 1.6781343558548683,
             'packet_redundancy': 9.20601913424517,
             'permeability': 0.8459393376038151,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.2835}}
